In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# build MultiScaleDeformableAttention
!cd /content/drive/MyDrive/models/dino/ops
!export TORCH_CUDA_ARCH_LIST="7.5;8.0;8.6;8.9"   # 7.5 for grand-challenge's online GPU (T4), 8.6 for 3090
!python /content/drive/MyDrive/models/dino/ops/setup.py install

running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [ ]:
!pip install SimpleITK fvcore ultralytics addict yapf==0.31.0 pycocotools

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/MultiScaleDeformableAttention-1.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/")

In [ ]:
import glob
import numpy as np
import os
import cv2
from tqdm import tqdm
import sys

import json
import torch
from PIL import Image
import SimpleITK as sitk
from torchvision import transforms as T
from torchvision.transforms import functional as F
from torchvision.ops import box_iou
from detectron2.config import get_cfg
from detectron2.engine.defaults import DefaultPredictor
from models.diffusiondet import DiffusionDetDatasetMapper, add_diffusiondet_config, DiffusionDetWithTTA
from models.diffusiondet.util.model_ema import add_model_ema_configs
from ultralytics import YOLO
from models.unet.utils import load_unet, load_seunet
from util.slconfig import SLConfig

from util.weighted_boxes_fusion import weighted_boxes_fusion
import models.dino.datasets.transforms as DT
import argparse
import torch.serialization
torch.serialization.add_safe_globals([argparse.Namespace])
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

quadrant_remap = {
    0: 1,
    1: 0,
    2: 2,
    3: 3,
}

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

def load_custom_json(path, is_gt=True):
    """
    Convert Dentex-style JSON to COCO format for quadrants.
    """
    with open(path, 'r') as f:
        data = json.load(f)

    images = []
    annotations = []
    categories = [{"id": i, "name": str(i)} for i in range(4)]  # quadrants 0–3
    ann_id = 1
    image_ids_seen = {}
    next_img_id = 1

    for box in data["boxes"]:
        # Extract quadrant id
        try:
            q = int(box["name"])
        except ValueError:
            q = int(box["name"].split("-")[0])

        # Bounding box from corners
        xs = [c[0] for c in box["corners"]]
        ys = [c[1] for c in box["corners"]]
        x, y, w, h = min(xs), min(ys), max(xs) - min(xs), max(ys) - min(ys)

        # Assign image_id
        img_key = box.get("image_id") or box.get("image_name") or "img_unknown"
        if img_key not in image_ids_seen:
            image_ids_seen[img_key] = next_img_id
            images.append({
                "id": next_img_id,
                "file_name": f"{img_key}.png",
                "width": 2048,
                "height": 1024
            })
            next_img_id += 1
        image_id = image_ids_seen[img_key]

        ann = {
            "id": ann_id,
            "image_id": image_id,
            "category_id": q,
            "bbox": [x, y, w, h],
            "area": w * h,
            "iscrowd": 0
        }
        if not is_gt:
            ann["score"] = box.get("probability", 1.0)

        annotations.append(ann)
        ann_id += 1

    coco_dict = {
        "info": {"description": "Dentex Quadrant Dataset"},
        "licenses": [],
        "images": images,
        "annotations": annotations,
        "categories": categories
    }
    return coco_dict

In [ ]:
class D2DetectionPredictor:
    def __init__(self, cfg, threshold=0.5) -> None:
        self.predictor = DefaultPredictor(cfg)
        self.threshold = threshold

    def predict(self, image):
        predictions = self.predictor(image)
        instances = predictions["instances"]
        return instances[instances.scores > self.threshold]

In [ ]:
class DinoDetectionPredictor:
    def __init__(self, args, model_checkpoint_path, score_threshold, cuda=True, mean=None, std=None) -> None:
        model, criterion, postprocessors = build_model_main(args)
        checkpoint = torch.load(model_checkpoint_path, map_location="cpu")
        model.load_state_dict(checkpoint["model"])
        self.cuda = cuda
        if cuda:
            model = model.cuda()
        model.eval()
        self.model = model
        self.postprocessors = postprocessors

        self.transform = DT.Compose(
            [
                DT.RandomResize([800], max_size=1333),
                DT.ToTensor(),
                DT.Normalize(
                    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
                ),  # the origin config, maybe different from the dentex dataset
            ]
        )
        self.score_threshold = score_threshold

    def predict(self, image: np.ndarray):
        image_shape = image.shape[:2]
        image = Image.fromarray(image)
        image, _ = self.transform(image, None)
        if self.cuda:
            image = image.cuda()
        outputs = self.model(image.unsqueeze(0))
        scale = torch.tensor([image_shape])
        if self.cuda:
            scale = scale.cuda()
        output = self.postprocessors["bbox"](outputs, scale)[0]
        scores = output["scores"]
        labels = output["labels"]
        boxes = output["boxes"]
        select_mask = scores > self.score_threshold

        scores = scores[select_mask]
        labels = labels[select_mask]
        boxes = boxes[select_mask]

        return {
            "scores": scores,
            "labels": labels,
            "boxes": boxes,
        }

In [ ]:
class SegmentationPredictor:
    def __init__(self, model, mean=0.458, std=0.173, cuda=True) -> None:
        self.model = model
        self.model.eval()
        self.cuda = cuda
        self.mean = mean  # default mean and std here are caculated from dentex dataset
        self.std = std

    def predict(self, image: np.ndarray) -> torch.Tensor:
        origin_shape = image.shape
        image = Image.fromarray(image).resize((256, 256))
        image = F.to_tensor(image)
        image = F.normalize(image, [self.mean], [self.std])
        if self.cuda:
            image = image.cuda()
        image = image.unsqueeze(0)

        predictions = self.model(image)
        predictions = predictions.squeeze(0)
        predictions = torch.argmax(predictions, dim=0, keepdim=True)
        predictions = F.resize(predictions, origin_shape, F.InterpolationMode.NEAREST)
        return predictions.squeeze(0)

In [ ]:
def label_mask_to_bbox(mask: np.ndarray):
    """
    convert segmentation mask to bbox, only keep the largest connected component for each label
    mask: (H, W)
    return: bbox_dict, key is label, value is bbox (x, y, w, h)
    """
    bbox_dict = {}
    for label in np.unique(mask):
        if label == 0:
            continue
        label_mask = mask == label
        label_mask = label_mask.astype(np.uint8)
        contours, hierarchy = cv2.findContours(label_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
        if len(contours) == 0:
            continue
        bbox = cv2.boundingRect(contours[0])
        bbox_dict[label] = bbox
    return bbox_dict

In [ ]:
def build_model_main(args):
    from models.registry import MODULE_BUILD_FUNCS

    assert args.modelname in MODULE_BUILD_FUNCS._module_dict
    build_func = MODULE_BUILD_FUNCS.get(args.modelname)
    model, criterion, postprocessors = build_func(args)
    return model, criterion, postprocessors

In [ ]:
def calc_iou(box1, box2):
    """
    calculate iou between two boxes
        box: (x1, y1, x2, y2)
    """

    box1_tensor = torch.tensor(box1).float()
    box2_tensor = torch.tensor(box2).float()
    return box_iou(box1_tensor.unsqueeze(0), box2_tensor.unsqueeze(0)).item()

In [ ]:
def format_result_json_obj(disease_instances):
    """
    format disease_instances to fianl json object
    """
    result = {
        "name": "Regions of interest",
        "type": "Multiple 2D bounding boxes",
        "boxes": [],
        "version": {"major": 1, "minor": 0},
    }
    for disease_instance in disease_instances:
        bbox = disease_instance["bbox"]
        disease_id = disease_instance["disease_id"]
        enumeration_id = disease_instance["enumeration_id"]

        category_str = f"{(enumeration_id - 1) // 8} - {(enumeration_id - 1) % 8} - {disease_id}"
        x1, y1, x2, y2 = bbox
        image_id = int(disease_instance["image_id"])
        corners = [[x1, y1, image_id], [x1, y2, image_id], [x2, y1, image_id], [x2, y2, image_id]]
        box_meta = {
            "name": category_str,
            "corners": corners,
            "probability": disease_instance["score"],
        }
        result["boxes"].append(box_meta)

    return result

In [ ]:
def filter_duplicated_disease_instances(disease_instances):
    # only keep the highest score instance for the same label on the same image
    filtered_disease_instances = []
    for disease_instance in disease_instances:
        duplicate = False
        for filtered_disease_instance in filtered_disease_instances:
            if (
                disease_instance["image_id"] == filtered_disease_instance["image_id"]
                and disease_instance["disease_id"] == filtered_disease_instance["disease_id"]
                and disease_instance["enumeration_id"] == filtered_disease_instance["enumeration_id"]
            ):
                duplicate = True
                break

        if not duplicate:
            filtered_disease_instances.append(disease_instance)
        else:
            if disease_instance["score"] > filtered_disease_instance["score"]:
                disease_instance["bbox"] = filtered_disease_instance["bbox"]
                filtered_disease_instances.remove(filtered_disease_instance)
                filtered_disease_instances.append(disease_instance)

    print(f"before: {len(disease_instances)} instances, after: {len(filtered_disease_instances)} instances")
    return filtered_disease_instances

In [ ]:
def filter_disease_instances_by_prior(disease_instances):
    # using some prior knowledge to filter disease instances

    filtered_disease_instances = []
    for disease_instance in disease_instances:
        disease_id = disease_instance["disease_id"]
        enumeration_id = disease_instance["enumeration_id"]

        # 1. impacted tooth should only appear on enumeration_id % 8 == 7
        # if it is close to, adjust it, otherwise filter it
        if disease_id == 0:
            if enumeration_id % 8 == 7:
                disease_instance["enumeration_id"] = enumeration_id + 1
                print(f"found an impacted tooth with enumeration_id {enumeration_id}, adjust to {enumeration_id + 1}")
            elif enumeration_id % 8 != 0:
                print(f"found an impacted tooth with enumeration_id {enumeration_id}, filter it")
                continue
            else:
                ...

        # 2. when caries and deep caries appear on the same tooth, only keep the deep caries
        if disease_id in [1, 3]:
            duplicate = False
            for filtered_disease_instance in filtered_disease_instances:
                if not (
                    disease_instance["image_id"] == filtered_disease_instance["image_id"]
                    and disease_instance["enumeration_id"] == filtered_disease_instance["enumeration_id"]
                ):
                    continue

                if filtered_disease_instance["disease_id"] in [1, 3]:
                    filtered_disease_instance["disease_id"] = 3
                    filtered_disease_instance["score"] = max(
                        filtered_disease_instance["score"], disease_instance["score"]
                    )
                    duplicate = True
                    print(
                        f"found a caries and a deep caries on the same tooth (image id: {disease_instance['image_id']}, enumeration id: {disease_instance['enumeration_id']}), only keep the deep caries"
                    )
                    break

            if duplicate:
                continue

        filtered_disease_instances.append(disease_instance)
    return filtered_disease_instances

In [ ]:
def _normalize_xyxy(boxes, W, H):
    """
    boxes: list[list[4]] or np.ndarray in absolute xyxy.
    Returns: list[list[4]] normalized [0,1].
    Handles empty and 1D inputs safely.
    """
    if boxes is None or len(boxes) == 0:
        return []
    arr = np.asarray(boxes, dtype=float)
    if arr.ndim == 1:
        arr = arr.reshape(1, 4)
    arr[:, [0, 2]] /= float(W)
    arr[:, [1, 3]] /= float(H)
    return arr.tolist()

def _denormalize_xyxy(boxes, W, H):
    if boxes is None or len(boxes) == 0:
        return []
    arr = np.asarray(boxes, dtype=float)
    if arr.ndim == 1:
        arr = arr.reshape(1, 4)
    arr[:, [0, 2]] *= float(W)
    arr[:, [1, 3]] *= float(H)
    return arr.tolist()


# **Old code**

In [ ]:
with torch.no_grad():
    confidence_threshold = 0.25
    cuda = True
    iou_match_threshold = 0.4
    container = True
    if container:
        input_image_dir = "/content/drive/MyDrive/dentex_test/input"
        output_json_path = "/content/drive/MyDrive/results/abnormal-teeth-detection.json"
    else:
        input_image_dir = "./"
        output_json_path = "./results/abnormal-teeth-detection.json"
    # load models
    quadrant_cfg = get_cfg()
    add_diffusiondet_config(quadrant_cfg)
    add_model_ema_configs(quadrant_cfg)
    quadrant_cfg.merge_from_file("/content/drive/MyDrive/configs/diffdet/diffdet.dentex.swinbase.quadrant.yaml")
    quadrant_cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/output_diffdet_quadrant/model_0029999.pth"
    quadrant_cfg.MODEL.RETINANET.SCORE_THRESH_TEST = confidence_threshold
    quadrant_cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
    quadrant_cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = confidence_threshold
    quadrant_cfg.freeze()
    quadrant_predictor = D2DetectionPredictor(quadrant_cfg)

    disease_dino_model_config_path = "/content/drive/MyDrive/configs/dino/DINO_4scale_swin_cls4.py"
    disease_dino_model_checkpoint_path = "/content/drive/MyDrive/output_swin_disease_all/checkpoint0027.pth"
    disease_dino_cfg = SLConfig.fromfile(disease_dino_model_config_path)
    disease_dino_cfg.device = "cuda" if cuda else "cpu"
    disease_dino_predictor = DinoDetectionPredictor(
        disease_dino_cfg, disease_dino_model_checkpoint_path, confidence_threshold, cuda=cuda
    )

    disease_yolo_predictor = YOLO("/content/drive/MyDrive/runs_Disease_all/detect/train/weights/best.pt")

    enumeration_detector_config_path = "/content/drive/MyDrive/configs/dino/DINO_4scale_cls32.py"
    enumeration_detector_checkpoint_path = "/content/drive/MyDrive/output_dino_res50_enum32/checkpoint0024.pth"
    enumeration_cfg = SLConfig.fromfile(enumeration_detector_config_path)
    enumeration_cfg.device = "cuda" if cuda else "cpu"
    enumeration_predictor = DinoDetectionPredictor(
        enumeration_cfg, enumeration_detector_checkpoint_path, 0.3, cuda=cuda
    )

    enumeration9_segmentation_models = [
        load_unet("/content/drive/MyDrive/output_unet_enum9_07-26_10-25/epoch_34_loss_0.7818149849772453.pth", 10, cuda=cuda),  # out_channels include background
        load_unet("/content/drive/MyDrive/output_unet_enum9_07-26_10-25/epoch_38_loss_0.7756352759897709.pth", 10, cuda=cuda),
        load_seunet("/content/drive/MyDrive/output_seunet_enum9_07-26_17-00/epoch_45_loss_0.721814745105803.pth", 10, cuda=cuda),
    ]

    enumeration32_segmentation_models = [
        load_unet("/content/drive/MyDrive/output_unet_enum32_07-25_13-34/epoch_166_loss_0.5442415252327919.pth", 33, cuda=cuda),
        load_seunet("/content/drive/MyDrive/output_seunet_enum32_07-25_16-46/epoch_148_loss_0.5509927049279213.pth", 33, cuda=cuda),
        load_seunet("/content/drive/MyDrive/output_seunet_enum32_07-25_16-46/epoch_201_loss_0.5455089658498764.pth", 33, cuda=cuda),
    ]

    enumeration9_segmentation_predictors = [
        SegmentationPredictor(model, cuda=cuda) for model in enumeration9_segmentation_models
    ]

    enumeration32_segmentation_predictors = [
        SegmentationPredictor(model, cuda=cuda) for model in enumeration32_segmentation_models
    ]


use_checkpoint!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
# --- Load COCO Metadata ---
with open("/content/drive/MyDrive/dentex_val/validation_triple.json", "r") as f:
    coco = json.load(f)
list_ids_final = []
for image_info in coco["images"]:
    list_ids_final.append({
        "file_name": image_info["file_name"],
        "height": image_info["height"],
        "width": image_info["width"],
        "id": image_info["id"]
    })
# --- Load PNG images into a dictionary ---
input_image_dir = "/content/drive/MyDrive/dentex_val/validation_data/quadrant_enumeration_disease/xrays"  # update if needed
image_array = {}
for item in tqdm(list_ids_final):
    img_path = os.path.join(input_image_dir, item["file_name"])
    img = cv2.imread(img_path)  # BGR
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")
    image_array[item["id"]] = img
# --- Inference ---
with torch.no_grad():
    all_disease_instances = []

    image_count = len(list_ids_final)
    list_ids = list_ids_final
    image_type = "test"

    print(f"task type: {image_type}")
    for entry in tqdm(list_ids):
        image_id = entry["id"]
        image_name = entry["file_name"]
        image_height = entry["height"]
        image_width = entry["width"]

        print(f"processing {image_name}, id {image_id}...")

        image_rgb_arr = image_array[image_id]
        image_rgb_arr = image_rgb_arr[:image_height, :image_width, :]  # Crop just in case
        image_bgr_arr = image_rgb_arr[:, :, ::-1]  # RGB → BGR
        image_gray_arr = cv2.cvtColor(image_bgr_arr, cv2.COLOR_BGR2GRAY)

        enumeration_bboxes_each_seg_model: list[dict[int, list[int]]] = []  # list[dict[enumeration_id, bbox]]
        enumeration_result_by_detector = {}
        # 1. detect quadrants
        quadrant_instances = quadrant_predictor.predict(image_bgr_arr)
        # select highest score instance for each quadrant
        quadrant_bboxes_dict = {}
        quadrant_scores_dict = {}
        for quadrant_instance_id in range(len(quadrant_instances)):
            quadrant_bbox = quadrant_instances.pred_boxes.tensor[quadrant_instance_id].tolist()
            quadrant_bbox = [int(x) for x in quadrant_bbox]
            quadrant_id = quadrant_instances.pred_classes[quadrant_instance_id].item()
            quadrant_id = quadrant_remap[quadrant_id]  # 0, 1, 2, 3 -> 1, 0, 2, 3
            quadrant_score = quadrant_instances.scores[quadrant_instance_id].item()
            if quadrant_id not in quadrant_bboxes_dict or quadrant_score > quadrant_scores_dict[quadrant_id]:
                quadrant_bboxes_dict[quadrant_id] = quadrant_bbox
                quadrant_scores_dict[quadrant_id] = quadrant_score

        # 2. crop each quadrant and predict 9 class segmentation
        for quadrant_id, quadrant_bbox in quadrant_bboxes_dict.items():
            quadrant_image_gray_arr = image_gray_arr[
                quadrant_bbox[1] : quadrant_bbox[3], quadrant_bbox[0] : quadrant_bbox[2]
            ]
            for quadrant_enumeration_predictor in enumeration9_segmentation_predictors:
                quadrant_enumeration_prediction_mask = quadrant_enumeration_predictor.predict(quadrant_image_gray_arr)
                quadrant_enumeration_prediction_mask = quadrant_enumeration_prediction_mask.cpu().numpy()
                quadrant_enumeratino_prediction_bbox_dict = label_mask_to_bbox(quadrant_enumeration_prediction_mask)

                enumeration_bboxes_each_seg_model.append({})
                # convert local bbox to global bbox
                for quadrant_enumeration_id, enumeration_bbox in quadrant_enumeratino_prediction_bbox_dict.items():
                    if quadrant_enumeration_id == 9:
                        continue

                    enumeration_bbox = [  # local to global and meanwhile xywh to xyxy
                        enumeration_bbox[0] + quadrant_bbox[0],
                        enumeration_bbox[1] + quadrant_bbox[1],
                        enumeration_bbox[0] + quadrant_bbox[0] + enumeration_bbox[2],
                        enumeration_bbox[1] + quadrant_bbox[1] + enumeration_bbox[3],
                    ]

                    enumeration_id = quadrant_id * 8 + quadrant_enumeration_id
                    enumeration_bboxes_each_seg_model[-1].update({enumeration_id: enumeration_bbox})

        # 3. predict 32 class segmentation
        for enumeration_predictors in enumeration32_segmentation_predictors:
            enumeration_prediction_mask = enumeration_predictors.predict(image_gray_arr)
            enumeration_prediction_mask = enumeration_prediction_mask.cpu().numpy()
            enumeration_prediction_bbox_dict = label_mask_to_bbox(enumeration_prediction_mask)

            enumeration_bboxes_each_seg_model.append({})
            for enumeration_id, enumeration_bbox in enumeration_prediction_bbox_dict.items():
                enumeration_bboxes_each_seg_model[-1].update({enumeration_id: enumeration_bbox})

        enumeration_detector_prediction = enumeration_predictor.predict(image_rgb_arr)
        for enumeration_instance_id in range(len(enumeration_detector_prediction["boxes"])):
            enumeration_bbox = enumeration_detector_prediction["boxes"][enumeration_instance_id].tolist()
            # enumeration_bbox = [int(x) for x in enumeration_bbox]
            enumeration_score = enumeration_detector_prediction["scores"][enumeration_instance_id].item()
            enumeration_label = enumeration_detector_prediction["labels"][enumeration_instance_id].item() + 1

            enumeration_result_by_detector.update(
                {enumeration_label: {"bbox": enumeration_bbox, "score": enumeration_score}}
            )

        # 4. detect disease
        disease_dino_instances = disease_dino_predictor.predict(image_rgb_arr)
        disease_dino_boxes = []
        disease_dino_labels = []
        disease_dino_scores = []

        for disease_instance_id in range(len(disease_dino_instances["boxes"])):
            disease_bbox = disease_dino_instances["boxes"][disease_instance_id].tolist()
            # disease_bbox = [int(x) for x in disease_bbox]
            disease_score = disease_dino_instances["scores"][disease_instance_id].item()
            disease_label = disease_dino_instances["labels"][disease_instance_id].item()

            disease_dino_boxes.append(disease_bbox)
            disease_dino_labels.append(disease_label)
            disease_dino_scores.append(disease_score)

        disease_yolo_instances = disease_yolo_predictor.predict(image_bgr_arr, conf=confidence_threshold)[0].boxes
        disease_yolo_boxes = []
        disease_yolo_labels = []
        disease_yolo_scores = []

        for disease_instance_id in range(len(disease_yolo_instances)):
            disease_bbox = disease_yolo_instances.xyxy[disease_instance_id].tolist()
            # disease_bbox = [int(x) for x in disease_bbox]
            disease_score = disease_yolo_instances.conf[disease_instance_id].item()
            disease_label = int(disease_yolo_instances.cls[disease_instance_id].item())

            disease_yolo_boxes.append(disease_bbox)
            disease_yolo_labels.append(disease_label)
            disease_yolo_scores.append(disease_score)

        # normalize disease bbox for weighted_boxes_fusion

        # ---------------- normalize disease bbox for weighted_boxes_fusion ----------------
        disease_dino_boxes_norm = _normalize_xyxy(disease_dino_boxes, image_width, image_height)
        disease_yolo_boxes_norm = _normalize_xyxy(disease_yolo_boxes, image_width, image_height)

        # Make sure scores/labels lengths match boxes (truncate if any lib returned mismatched lengths)
        if len(disease_dino_scores) != len(disease_dino_boxes_norm):
            n = min(len(disease_dino_scores), len(disease_dino_boxes_norm))
            disease_dino_boxes_norm = disease_dino_boxes_norm[:n]
            disease_dino_scores     = disease_dino_scores[:n]
            disease_dino_labels     = disease_dino_labels[:n]

        if len(disease_yolo_scores) != len(disease_yolo_boxes_norm):
            n = min(len(disease_yolo_scores), len(disease_yolo_boxes_norm))
            disease_yolo_boxes_norm = disease_yolo_boxes_norm[:n]
            disease_yolo_scores     = disease_yolo_scores[:n]
            disease_yolo_labels     = disease_yolo_labels[:n]

        # If both models have zero boxes, skip fusion for this image
        if (len(disease_dino_boxes_norm) == 0) and (len(disease_yolo_boxes_norm) == 0):
            disease_instances_list = []
        else:
            # ---------------- fuse disease bbox ----------------
            fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
                [disease_dino_boxes_norm, disease_yolo_boxes_norm],
                [disease_dino_scores,     disease_yolo_scores],
                [disease_dino_labels,     disease_yolo_labels],
                weights=[2, 1],
                iou_thr=0.50,
                skip_box_thr=0.0,   # keep low-conf boxes for fusion; tune if needed
            )

            # ---------------- denormalize disease bbox ----------------
            fused_boxes = _denormalize_xyxy(fused_boxes, image_width, image_height)

            # Build instances
            disease_instances_list = []
            for b, s, l in zip(fused_boxes, fused_scores, fused_labels):
                disease_instances_list.append(
                    {"bbox": b, "score": float(s), "disease_id": int(l), "image_id": image_id}
                )

        disease_instances_list = []
        for disease_instance_id in range(len(fused_boxes)):
            disease_bbox = fused_boxes[disease_instance_id]
            disease_score = fused_scores[disease_instance_id]
            disease_label = int(fused_labels[disease_instance_id])

            disease_instances_list.append(
                {
                    "bbox": disease_bbox,
                    "score": disease_score,
                    "disease_id": disease_label,
                    "image_id": image_id,
                }
            )

        print(f"image {image_id} has {len(disease_instances_list)} disease instances")

        # 5. for each diease instance, match the most possible enumeration_id
        # voting by iou from many segmentation and detection models
        filtered_disease_instances_list = []
        for disease_instance in disease_instances_list:
            disease_bbox = disease_instance["bbox"]
            disease_id = disease_instance["disease_id"]

            # record the accumulated iou for each enumeration_id
            enumeration_id_iou_dict = {i: 0 for i in range(1, 33)}
            for enumeration_bboxes in enumeration_bboxes_each_seg_model:
                for enumeration_id, enumeration_bbox in enumeration_bboxes.items():
                    iou = calc_iou(disease_bbox, enumeration_bbox)
                    if iou > iou_match_threshold:
                        enumeration_id_iou_dict[enumeration_id] += iou

            # higher weight for detection result
            for enumeration_id, enumeration_result in enumeration_result_by_detector.items():
                enumeration_bbox = enumeration_result["bbox"]
                iou = calc_iou(disease_bbox, enumeration_bbox)
                if iou > iou_match_threshold:
                    enumeration_id_iou_dict[enumeration_id] += iou * 3 * enumeration_result["score"]

            # select the enumeration_id with highest accumulated iou
            max_iou = 0
            max_iou_enumeration_id = 0
            for enumeration_id, accumulated_iou in enumeration_id_iou_dict.items():
                if accumulated_iou > max_iou:
                    max_iou = accumulated_iou
                    max_iou_enumeration_id = enumeration_id

            if max_iou_enumeration_id == 0:
                print(
                    f"waring, image {image_id} disease: {disease_id}, bbox: {disease_bbox} has no enumeration_id, ingore it"
                )
            else:
                disease_instance["enumeration_id"] = max_iou_enumeration_id
                filtered_disease_instances_list.append(disease_instance)
        all_disease_instances.extend(filtered_disease_instances_list)


100%|██████████| 50/50 [00:04<00:00, 12.19it/s]


task type: test


  0%|          | 0/50 [00:00<?, ?it/s]

processing val_15.png, id 1...

0: 320x640 3 Impacteds, 3 Cariess, 12.4ms
Speed: 29.3ms preprocess, 12.4ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
image 1 has 12 disease instances


  2%|▏         | 1/50 [00:02<02:13,  2.72s/it]

...

 96%|█████████▌| 48/50 [01:29<00:03,  1.81s/it]

processing val_22.png, id 49...

0: 320x640 1 Impacted, 4 Cariess, 1 Deep Caries, 14.2ms
Speed: 30.1ms preprocess, 14.2ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)
image 49 has 10 disease instances


 98%|█████████▊| 49/50 [01:30<00:01,  1.83s/it]

processing val_0.png, id 50...

0: 352x640 1 Caries, 13.0ms
Speed: 33.1ms preprocess, 13.0ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 640)
image 50 has 9 disease instances


100%|██████████| 50/50 [01:33<00:00,  1.86s/it]


In [ ]:
with torch.no_grad():
    all_disease_instances = filter_duplicated_disease_instances(all_disease_instances)
    all_disease_instances = filter_disease_instances_by_prior(all_disease_instances)
    # 6. save result
    print("saving result...")
    result_json_obj = format_result_json_obj(all_disease_instances)
    with open(output_json_path, "w") as f:
        json.dump(result_json_obj, f, indent=4)

before: 400 instances, after: 398 instances
found a caries and a deep caries on the same tooth (image id: 7, enumeration id: 15), only keep the deep caries
found a caries and a deep caries on the same tooth (image id: 21, enumeration id: 23), only keep the deep caries
found a caries and a deep caries on the same tooth (image id: 21, enumeration id: 7), only keep the deep caries
found a caries and a deep caries on the same tooth (image id: 25, enumeration id: 30), only keep the deep caries
found a caries and a deep caries on the same tooth (image id: 25, enumeration id: 14), only keep the deep caries
found an impacted tooth with enumeration_id 23, adjust to 24
found a caries and a deep caries on the same tooth (image id: 30, enumeration id: 7), only keep the deep caries
found a caries and a deep caries on the same tooth (image id: 30, enumeration id: 8), only keep the deep caries
found a caries and a deep caries on the same tooth (image id: 32, enumeration id: 13), only keep the deep ca

In [ ]:
import json

# Load the original file
with open("/content/drive/MyDrive/dentex_val/GT.json", "r") as f:
    data = json.load(f)

# Prepare the three output structures
q_data = {k: [] if isinstance(v, list) else v for k, v in data.items()}
e_data = {k: [] if isinstance(v, list) else v for k, v in data.items()}
d_data = {k: [] if isinstance(v, list) else v for k, v in data.items()}

for box in data["boxes"]:
    q, e, d = box["name"].split(" - ")
    # Quadrant only
    q_box = box.copy()
    q_box["name"] = q
    q_data["boxes"].append(q_box)

    # Enumeration only
    e_box = box.copy()
    e_box["name"] = e
    e_data["boxes"].append(e_box)

    # Disease only
    d_box = box.copy()
    d_box["name"] = d
    d_data["boxes"].append(d_box)

# Save the three files
with open("/content/drive/MyDrive/dentex_val/GT_quadrant.json", "w") as f:
    json.dump(q_data, f, indent=4)

with open("/content/drive/MyDrive/dentex_val/GT_enum.json", "w") as f:
    json.dump(e_data, f, indent=4)

with open("/content/drive/MyDrive/dentex_val/GT_disease.json", "w") as f:
    json.dump(d_data, f, indent=4)

print("Files saved: GT_quadrant.json, GT_enum.json, GT_disease.json")


Files saved: GT_quadrant.json, GT_enum.json, GT_disease.json


In [ ]:
import json

# Load the original file
with open("/content/drive/MyDrive/results/abnormal-teeth-detection.json", "r") as f:
    data = json.load(f)

# Prepare the three output structures
q_data = {k: [] if isinstance(v, list) else v for k, v in data.items()}
e_data = {k: [] if isinstance(v, list) else v for k, v in data.items()}
d_data = {k: [] if isinstance(v, list) else v for k, v in data.items()}

for box in data["boxes"]:
    q, e, d = box["name"].split(" - ")
    # Quadrant only
    q_box = box.copy()
    q_box["name"] = q
    q_data["boxes"].append(q_box)

    # Enumeration only
    e_box = box.copy()
    e_box["name"] = e
    e_data["boxes"].append(e_box)

    # Disease only
    d_box = box.copy()
    d_box["name"] = d
    d_data["boxes"].append(d_box)

# Save the three files
with open("/content/drive/MyDrive/results/pred_quadrant.json", "w") as f:
    json.dump(q_data, f, indent=4)

with open("/content/drive/MyDrive/results/pred_enum.json", "w") as f:
    json.dump(e_data, f, indent=4)

with open("/content/drive/MyDrive/results/pred_disease.json", "w") as f:
    json.dump(d_data, f, indent=4)

print("Files saved: pred_quadrant.json, pred_enum.json, pred_disease.json")


Files saved: pred_quadrant.json, pred_enum.json, pred_disease.json


In [ ]:
# --- Paths ---
quadrant_gt_path = "/content/drive/MyDrive/dentex_val/GT_quadrant.json"
quadrant_pred_path = "/content/drive/MyDrive/results/pred_quadrant.json"

# --- Convert ---
quadrant_gt_coco = load_custom_json(quadrant_gt_path, is_gt=True)
quadrant_pred_coco_list = load_custom_json(quadrant_pred_path, is_gt=False)["annotations"]

# --- Save ---
with open("quadrant_gt_coco.json", "w") as f:
    json.dump(quadrant_gt_coco, f)
with open("quadrant_pred_coco.json", "w") as f:
    json.dump(quadrant_pred_coco_list, f)
print('*'*28, end='')
print('Quadrant predictions',end='')
print('*'*28)
# --- Evaluate ---
coco_gt = COCO("quadrant_gt_coco.json")
coco_dt = coco_gt.loadRes("quadrant_pred_coco.json")
coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
#-------------------------------------------------------------------------------------------
print('*'*28, end='')
print('Enum predictions',end='')
print('*'*28)
# --- Paths ---
enum_gt_path = "/content/drive/MyDrive/dentex_val/GT_enum.json"
enum_pred_path = "/content/drive/MyDrive/results/pred_enum.json"

# --- Convert ---
enum_gt_coco = load_custom_json(enum_gt_path, is_gt=True)
enum_pred_coco_list = load_custom_json(enum_pred_path, is_gt=False)["annotations"]

# --- Save ---
with open("enum_gt_coco.json", "w") as f:
    json.dump(enum_gt_coco, f)
with open("enum_pred_coco.json", "w") as f:
    json.dump(enum_pred_coco_list, f)
# --- Evaluate ---
coco_gt = COCO("enum_gt_coco.json")
coco_dt = coco_gt.loadRes("enum_pred_coco.json")
coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
#-------------------------------------------------------------------------------------------
print('*'*28, end='')
print('disease predictions',end='')
print('*'*28)
# --- Paths ---
disease_gt_path = "/content/drive/MyDrive/dentex_val/GT_disease.json"
disease_pred_path = "/content/drive/MyDrive/results/pred_disease.json"

# --- Convert ---
disease_gt_coco = load_custom_json(disease_gt_path, is_gt=True)
disease_pred_coco_list = load_custom_json(disease_pred_path, is_gt=False)["annotations"]

# --- Save ---
with open("disease_gt_coco.json", "w") as f:
    json.dump(disease_gt_coco, f)
with open("disease_pred_coco.json", "w") as f:
    json.dump(disease_pred_coco_list, f)
# --- Evaluate ---
coco_gt = COCO("disease_gt_coco.json")
coco_dt = coco_gt.loadRes("disease_pred_coco.json")
coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()


****************************Quadrant predictions****************************
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.473
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.768
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.493
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.013
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=

In [ ]:
import os
import json
import cv2
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# ========= CONFIG =========
GT_JSON_PATH   = "/content/drive/MyDrive/dentex_val/GT.json"                              # change if needed
PRED_JSON_PATH = "/content/drive/MyDrive/results/abnormal-teeth-detection.json"        # change if needed
IMAGES_DIR     = "/content/drive/MyDrive/dentex_val/validation_data/quadrant_enumeration_disease/xrays"                                 # folder with val_0.png ... val_49.png
OUTPUT_DIR     = "./plots_gt_vs_pred"
PLOT_MAX_PER_IMAGE = None  # None = plot all boxes, or set an int to limit text clutter
os.makedirs(OUTPUT_DIR, exist_ok=True)
CONF_THRESH    = 0.25
# ===============

def load_boxes(json_path, conf_thresh=0.25):
    with open(json_path, "r") as f:
        data = json.load(f)
    out = {}
    for b in data.get("boxes", []):
        p = float(b.get("probability", 1.0))
        if p < conf_thresh:  # filter low conf
            continue
        corners = b.get("corners", [])
        if len(corners) != 4:
            continue
        xs = [float(c[0]) for c in corners]
        ys = [float(c[1]) for c in corners]
        ids = [int(c[2]) for c in corners]
        if len(set(ids)) != 1:
            continue
        img_id = ids[0]
        x1, x2 = min(xs), max(xs)
        y1, y2 = min(ys), max(ys)
        out.setdefault(img_id, []).append({
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "label": b.get("name", ""), "prob": p
        })
    return out

def _is_normalized(all_x, all_y):
    return (max(all_x) <= 1.2) and (max(all_y) <= 1.2)

def _try_scale_factor(all_x, all_y, W, H):
    """If many coords are bigger than image, guess a single scale factor."""
    sx = max(all_x) / max(W, 1)
    sy = max(all_y) / max(H, 1)
    s = max(sx, sy)
    # Only apply if it looks like a real scale (>1.2× and <10×)
    return s if s > 1.2 and s < 10 else None

def apply_scaling(boxes, W, H, norm=False, scale_factor=None):
    out = []
    for b in boxes:
        x1, y1, x2, y2 = b["x1"], b["y1"], b["x2"], b["y2"]
        if norm:
            x1, x2 = x1 * W, x2 * W
            y1, y2 = y1 * H, y2 * H
        elif scale_factor is not None:
            x1, x2 = x1 / scale_factor, x2 / scale_factor
            y1, y2 = y1 / scale_factor, y2 / scale_factor
        nb = dict(b); nb.update({"x1": x1, "y1": y1, "x2": x2, "y2": y2})
        out.append(nb)
    return out

def clip_boxes_to_image(boxes, W, H):
    clipped = []
    for b in boxes:
        x1 = max(0, min(W - 1, b["x1"]))
        y1 = max(0, min(H - 1, b["y1"]))
        x2 = max(0, min(W - 1, b["x2"]))
        y2 = max(0, min(H - 1, b["y2"]))
        if x2 > x1 and y2 > y1:
            nb = dict(b)
            nb.update({"x1": x1, "y1": y1, "x2": x2, "y2": y2})
            clipped.append(nb)
    return clipped

def draw_boxes(ax, boxes, color, prefix):
    for b in boxes:
        ax.add_patch(plt.Rectangle((b["x1"], b["y1"]),
                                   b["x2"] - b["x1"], b["y2"] - b["y2"],
                                   fill=False, edgecolor=color, linewidth=2))
        ax.text(b["x1"], max(0, b["y1"] - 5),
                f"{prefix}: {b['label']} ({b['prob']:.2f})",
                color='white', fontsize=7,
                bbox=dict(facecolor=color, alpha=0.6, edgecolor='none'))

# Load JSONs
gt_by_img   = load_boxes(GT_JSON_PATH,   CONF_THRESH)
pred_by_img = load_boxes(PRED_JSON_PATH, CONF_THRESH)
all_ids = sorted(set(gt_by_img.keys()) | set(pred_by_img.keys()))

for img_id in tqdm(all_ids, desc="Plotting"):
    img_name = f"val_{img_id}.png"
    img_path = os.path.join(IMAGES_DIR, img_name)
    img = cv2.imread(img_path)
    if img is None:
        print(f"[WARN] Missing {img_path}")
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W = img.shape[:2]

    gts   = gt_by_img.get(img_id, [])
    preds = pred_by_img.get(img_id, [])
    both  = gts + preds

    if not both:
        continue

    all_x = [c for b in both for c in (b["x1"], b["x2"])]
    all_y = [c for b in both for c in (b["y1"], b["y2"])]

    # 1) detect normalized; 2) otherwise try scale factor; 3) else use raw
    norm = _is_normalized(all_x, all_y)
    scale_factor = None if norm else _try_scale_factor(all_x, all_y, W, H)

    if norm:
        print(f"[val_{img_id}] coords look normalized -> scaling to pixels")
    elif scale_factor:
        print(f"[val_{img_id}] applying scale factor ~{scale_factor:.2f} to fit image")
    else:
        # still warn if extreme
        if max(all_x) > W * 1.2 or max(all_y) > H * 1.2:
            print(f"[val_{img_id}] WARNING: coords exceed image; clipping will be applied")

    gts_scaled   = apply_scaling(gts,   W, H, norm=norm, scale_factor=scale_factor)
    preds_scaled = apply_scaling(preds, W, H, norm=norm, scale_factor=scale_factor)

    gts_final   = clip_boxes_to_image(gts_scaled,   W, H)
    preds_final = clip_boxes_to_image(preds_scaled, W, H)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(img)
    ax.set_xlim(0, W)
    ax.set_ylim(H, 0)   # y-down
    ax.set_aspect('equal')
    ax.axis('off')

    draw_boxes(ax, gts_final,   "lime", "GT")
    draw_boxes(ax, preds_final, "red",  "Pred")

    out_path = os.path.join(OUTPUT_DIR, f"gt_pred_{img_name}")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150, bbox_inches="tight")
    plt.close(fig)

print(f"✅ Saved combined plots to {OUTPUT_DIR}")



Plotting: 100%|██████████| 49/49 [00:39<00:00,  1.24it/s]

[WARN] Missing /content/drive/MyDrive/dentex_val/validation_data/quadrant_enumeration_disease/xrays/val_50.png
✅ Saved combined plots to ./plots_gt_vs_pred


In [ ]:
!rm -r /content/plots_gt_vs_pred